###### Gender inference using different tools

1) gender guesser library - yes
2) genderize.io - yes 
3) gender API - not needed as of now
4) genderComputer - not needed as of now
5) NamSor - not needed as of now

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import ast
import re
from collections import Counter
!pip install gender-guesser
import gender_guesser.detector as gender
import requests
from concurrent.futures import ThreadPoolExecutor
# pip install git+https://github.com/namsor/namsor-python-sdk2.git

/Users/Isha/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Read CSV files from data folder and store in dataframes
hackathons_webscraped = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/hackathons_webscraped.csv')
hackathons = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/hackathons_modified.csv')
participants = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/participants_modified.csv')
participants_links = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/participants_links_modified.csv')
projects = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/projects_modified.csv')
participants_links_expanded = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/participants_links_expanded.csv')

/var/folders/_f/7jgfw7qx5jn8xl64g_9px9g00000gp/T/ipykernel_66897/11719672.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  participants = pd.read_csv('/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/participants_modified.csv')


In [3]:
participants_links_expanded.columns
# creating a new df with 'name' and 'cleaned_names' to process using the website itself
cleaned_names_only_df = participants_links_expanded[['cleaned_names']].copy()
# cleaned_names_only_df.to_csv('cleaned_names_only.csv')

### 1) gender guesser library

In [4]:
# testing gender-guesser library
detector = gender.Detector()

# Example
name = "Alex"
gender = detector.get_gender(name)
print(f"The inferred gender for {name} is {gender}.")

The inferred gender for Alex is male.


In [5]:
# Ensure you've imported the gender_guesser.detector and created an instance of the Detector
import gender_guesser.detector as gender
detector = gender.Detector()

# Function to apply the gender detector on each name
def infer_gender_from_name(full_name):
    # Assuming the first name is the first word in the 'name' column entry
    first_name = full_name.split()[0] if pd.notnull(full_name) else ''
    return detector.get_gender(first_name)

# Applying the function to the 'name' column of your DataFrame
participants_links_expanded['gender_guesser_lib'] = participants_links_expanded['cleaned_names'].apply(infer_gender_from_name)

# Display the first few entries to verify
participants_links_expanded[['cleaned_names', 'gender_guesser_lib']]

,cleaned_names,gender_guesser_lib
0,Craig Hiller,male
1,Andrew Sheleby,male
2,Evan Ho,male
3,Brian Lam,male
4,Lin Peng,female
...,...,...
56490,Mike Dimitrov,male
56491,Baiyun ( Ben) Gao,unknown
56492,Everet Siew,male
56493,Amulya Garimella,unknown


In [6]:
# Calculate the distribution of inferred genders
gender_distribution = participants_links_expanded['gender_guesser_lib'].value_counts()

# Display the distribution
print(gender_distribution)

gender_guesser_lib
male             24573
unknown          19223
female            8683
mostly_male       1910
andy              1227
mostly_female      879
Name: count, dtype: int64


Changing all the 'male' and 'mostly_male' values to 'm', and 'female' and 'mostly_female' values to 'f' and 'andy' values to 'unknown'

In [7]:
# Applying the function to the 'name' column of your DataFrame
participants_links_expanded['gender_guesser_lib'] = participants_links_expanded['cleaned_names'].apply(infer_gender_from_name)

# Replace the values as specified
replacement_dict = {
    'male': 'm',
    'mostly_male': 'm',
    'female': 'f',
    'mostly_female': 'f',
    'andy': 'unknown'
}

participants_links_expanded['gender_guesser_lib'] = participants_links_expanded['gender_guesser_lib'].replace(replacement_dict)

# Display the first few entries to verify
participants_links_expanded[['cleaned_names', 'gender_guesser_lib']]

,cleaned_names,gender_guesser_lib
0,Craig Hiller,m
1,Andrew Sheleby,m
2,Evan Ho,m
3,Brian Lam,m
4,Lin Peng,f
...,...,...
56490,Mike Dimitrov,m
56491,Baiyun ( Ben) Gao,unknown
56492,Everet Siew,m
56493,Amulya Garimella,unknown


In [8]:
# Calculate the distribution of inferred genders
gender_distribution = participants_links_expanded['gender_guesser_lib'].value_counts()

# Display the distribution
print(gender_distribution)

gender_guesser_lib
m          26483
unknown    20450
f           9562
Name: count, dtype: int64


#### Performance improved after using cleaned names.

### 2) genderize.io 

### The code given below is not fully functional for the genderize.io application, it was trial-an-experiment code to check if I can apply genderize.io through this method but later I found that I can upload the csv on their web tool itself to get the genders. Thus, seeing that it was a quicker and efficient process, I just used that method. However, I'm keeping the code below for future reference (if needed).

In [ ]:
import requests

def get_gender(name):
    url = "https://api.genderize.io/"
    params = {'name': name, 'apikey': '3516fa3d13cfd6c6f05cc7005f275c08'}  # Replace with your actual API key
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # HTTP error
    except Exception as err:
        print(f"An error occurred: {err}")  # Other error
    return "Error: API request unsuccessful."

# Example usage:
name_to_genderize = "Alex"
result = get_gender(name_to_genderize)
print(result)

In [ ]:
# example 2:
import requests

def get_gender(name):
    # Constructing the API URL
    url = f"https://api.genderize.io?name={name}"

    # Making the API request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extracting the gender and probability
        gender = data.get('gender', 'Not found')
        probability = data.get('probability', 0)
        return gender, probability
    else:
        return "Error", "Unable to fetch data"

# Example usage:
name = "Alice"
gender, probability = get_gender(name)
print(f"The predicted gender for {name} is {gender} with a probability of {probability}.")


In [ ]:
# Function to infer gender using genderize.io WITHOUT AN API KEY
def infer_gender_genderize_io(full_name):
    first_name = full_name.split()[0] if pd.notnull(full_name) else ''
    try:
        response = requests.get("https://api.genderize.io/", params={"name": first_name})
        response.raise_for_status()  # This will raise an exception for HTTP errors
        gender_data = response.json()
        return gender_data.get('gender', 'unknown')
    except requests.exceptions.RequestException as e:
        print(f"Request to Genderize.io failed: {e}")
        return 'unknown'

# Apply the gender inference function using genderize.io
participants_links_expanded['inferred_gender_genderize_io'] = participants_links_expanded['cleaned_names'].apply(infer_gender_genderize_io)

# Display the first few entries to verify
participants_links_expanded[['cleaned_names', 'inferred_gender_genderize_io']]

In [ ]:
participants_links_expanded.columns
len(participants_links_expanded.index)
print(participants_links_expanded.isna().sum())

In [ ]:
## Using the API key and applying to first 100 rows to check performance

import requests
import pandas as pd

def infer_gender_genderize_io(full_name, api_key):
    first_name = full_name.split()[0] if pd.notnull(full_name) else ''
    try:
        response = requests.get("https://api.genderize.io/", params={"name": first_name, "apikey": api_key})
        response.raise_for_status()  # This will raise an exception for HTTP errors
        gender_data = response.json()
        return gender_data.get('gender', 'unknown')
    except requests.exceptions.RequestException as e:
        print(f"Request to Genderize.io failed: {e}")
        return 'unknown'

# Apply the function to all names

api_key = '3516fa3d13cfd6c6f05cc7005f275c08'  # Replace this with your actual Genderize.io API key

# Apply the gender inference function using genderize.io to all rows
participants_links_expanded['inferred_gender_genderize_io'] = participants_links_expanded['cleaned_names'][:100].apply(lambda x: infer_gender_genderize_io(x, api_key))

# Display the first few entries to verify
print(participants_links_expanded[['cleaned_names', 'inferred_gender_genderize_io']])


#### Worked successfully on the first 100 rows, now need to apply this to the whole dataset

In [ ]:
participants_links_expanded.columns

In [ ]:
## trying batch processing 
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def batch_infer_gender(names, api_key):
    # This function sends a batch of names to Genderize.io in a single API request.
    url = "https://api.genderize.io/"
    params = {'apikey': api_key}
    params.update({f'name[{i}]': name for i, name in enumerate(names)})  # Proper batching
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return [res.get('gender', 'unknown') for res in response.json()]  # Process each response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ['unknown'] * len(names)  # Handle errors by returning 'unknown'

def apply_gender_inference(df, api_key):
    # Split names into chunks of 10
    chunks = [df[i:i+10] for i in range(0, len(df), 10)]
    results = []
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(batch_infer_gender, chunk, api_key) for chunk in chunks]
        for future in futures:
            results.extend(future.result())
    
    df['inferred_gender'] = results
    return df

# Sample the first 1000 entries for testing
api_key = '3516fa3d13cfd6c6f05cc7005f275c08'
sample_data = participants_links_expanded['cleaned_names'][:1000]  # Test on the first 1000 entries

# Apply the gender inference
genderized_results = apply_gender_inference(sample_data, api_key)
print(genderized_results.head())


In [ ]:
print(type(genderized_results))

In [ ]:
# the one before worked for 1000 enteries but gave it as a series so we need to fix it and get it in a df

from concurrent.futures import ThreadPoolExecutor
def batch_infer_gender(names, api_key):
    # This function sends a batch of names to Genderize.io in a single API request.
    url = "https://api.genderize.io/"
    params = {'apikey': api_key}
    params.update({f'name[{i}]': name for i, name in enumerate(names)})  # Proper batching
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return [res.get('gender', 'unknown') for res in response.json()]  # Process each response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ['unknown'] * len(names)  # Handle errors by returning 'unknown'

def apply_gender_inference(df, api_key):
    # Split names into chunks of 10
    chunks = [df[i:i+10] for i in range(0, len(df), 10)]
    results = []
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(batch_infer_gender, chunk, api_key) for chunk in chunks]
        for future in futures:
            results.extend(future.result())
    
    df['inferred_gender'] = results
    return df

# Sample the first 1000 entries for testing
api_key = '3516fa3d13cfd6c6f05cc7005f275c08'
sample_data = participants_links_expanded['cleaned_names'][:100]  # Test on the first 1000 entries

genderized_results = apply_gender_inference(sample_data, api_key)
genderized_results = genderized_results.to_frame(name='inferred_gender_io')
print(type(participants_links_expanded))  # Should be DataFrame
print(type(participants_links_expanded['cleaned_names']))  # Should be Series
print(type(genderized_results))  # Should be DataFrame after fix


In [ ]:
genderized_results

In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def batch_infer_gender(names, api_key):
    # This function sends a batch of names to Genderize.io in a single API request.
    url = "https://api.genderize.io/"
    params = {'apikey': api_key}
    params.update({f'name[{i}]': name for i, name in enumerate(names)})  # Proper batching
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return [res.get('gender', 'unknown') for res in response.json()]  # Process each response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ['unknown'] * len(names)  # Handle errors by returning 'unknown'

def apply_gender_inference(df, column_name, api_key):
    # Split names into chunks of 10
    names = df[column_name].tolist()
    chunks = [names[i:i+10] for i in range(0, len(names), 10)]
    results = []
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(batch_infer_gender, chunk, api_key) for chunk in chunks]
        for future in futures:
            results.extend(future.result())
    
    return pd.DataFrame({column_name: df[column_name], 'inferred_gender_io': results})

# Assuming participants_links_expanded is your main DataFrame
api_key = '3516fa3d13cfd6c6f05cc7005f275c08'
sample_data = participants_links_expanded  # Can sample the first 1000 entries for testing

# Ensure you're passing the DataFrame and the name of the column holding the names
genderized_results = apply_gender_inference(sample_data, 'cleaned_names', api_key)

print(type(participants_links_expanded))  # Should be DataFrame
print(type(participants_links_expanded['cleaned_names']))  # Should be Series
print(type(genderized_results))  # Should now be DataFrame after the fix
print(genderized_results.head())  # Display the first few rows to verify structure


In [ ]:
genderized_results

In [ ]:
# Merge the two DataFrames on the 'cleaned_names' column
participants_links_expanded = pd.merge(participants_links_expanded, genderized_results, on='cleaned_names', how='left')
participants_links_expanded.head()  # Check the first few rows to ensure the merge was successful

In [ ]:
#del participants_links_expanded['inferred_gender']
del participants_links_expanded['inferred_gender_genderize_io']
print(participants_links_expanded.isna().sum())
participants_links_expanded.columns

Issues:
1. cleaned_names has NAN values but names does not - need to fix or need to just use names column
- when i created the file, there was no issue with cleaned names, it was all good, so something went wrong elsewhere. 
2. which column to delete and use inferred_gender_io_x or inferred_gender_io_y - can check for disparity between the two and if there is no diff then delete either doesnt matter
    - same, can delete either

In [ ]:
# Calculate the distribution of inferred genders using genderize.io on CLEANED NAMES
gender_distribution = participants_links_expanded['inferred_gender_io_x'].value_counts()

# Display the distribution
print(gender_distribution)

Issues:
1. females > males - that changes everything AND it is totally different to the gender guess lib 
2. this has been applied to CLEANED NAMES which has 175K null values 
3. how does cleaned names have more null values than the total number of rows of 56k participants itself?

In [ ]:
## Apply the genderize function to just NAMES and check difference 
# Assuming participants_links_expanded is your main DataFrame
api_key = '3516fa3d13cfd6c6f05cc7005f275c08'
sample_data = participants_links_expanded  # Can sample the first 1000 entries for testing

# Ensure you're passing the DataFrame and the name of the column holding the names
genderized_results = apply_gender_inference(sample_data, 'name', api_key)

print(type(participants_links_expanded))  # Should be DataFrame
print(type(participants_links_expanded['name']))  # Should be Series
print(type(genderized_results))  # Should now be DataFrame after the fix
print(genderized_results.head())  # Display the first few rows to verify structure

In [ ]:
print(participants_links_expanded.isna().sum()) # checking for number of empty values
# Calculate the distribution of inferred genders using genderize.io on CLEANED NAMES
gender_distribution = participants_links_expanded['inferred_gender_io_x'].value_counts()

# Display the distribution
print(gender_distribution)

### After this, the first 600 names were run through the gender predictor GPT (OpenAI Chat-GPT 4 application) and also manually classified. See below for further analysis: 

## Comparing manual prediction, genderize.io, gender predictor gpt on the first 600 names

In [4]:
import pandas as pd

first_600_names_classified = pd.read_csv(r'/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/cleaned_names_only_first600names.csv', sep=';')
first_600_names_classified.columns

Index(['Unnamed: 0', 'cleaned_names', 'self_inference', 'genderize.io',
       'chatpgt gender predictor', 'Gender Probability', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')

In [5]:
#del first_600_names_classified[['Unnamed: 0', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']]
first_600_names_classified.columns
del first_600_names_classified['Unnamed: 0']
del first_600_names_classified['Unnamed: 6']
del first_600_names_classified['Unnamed: 7']
del first_600_names_classified['Unnamed: 8']

In [6]:
# Analyze only the first 600 rows
df_first_600 = first_600_names_classified.iloc[:600]

# Create a function to represent the unique sorted tuple
def unique_sorted_tuple(row):
    # Extract the tuple
    unique_comb = tuple(sorted(Counter(row).items()))
    return unique_comb

# Filter out rows where all values are the same
disparities = df_first_600[
    (df_first_600['self_inference'] != df_first_600['genderize.io']) |
    (df_first_600['self_inference'] != df_first_600['chatpgt gender predictor']) |
    (df_first_600['genderize.io'] != df_first_600['chatpgt gender predictor'])
]

# Apply unique sorted tuple function
disparities['unique_comb'] = disparities.apply(lambda x: unique_sorted_tuple([x['self_inference'], x['genderize.io'], x['chatpgt gender predictor']]), axis=1)

# Get unique combinations and their counts
unique_combinations = disparities['unique_comb'].value_counts().reset_index()
unique_combinations.columns = ['combination', 'count']

# Calculate the proportions
unique_combinations['proportion'] = unique_combinations['count'] / len(df_first_600)

# Display results
unique_combinations

/var/folders/_f/7jgfw7qx5jn8xl64g_9px9g00000gp/T/ipykernel_66533/1676829676.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disparities['unique_comb'] = disparities.apply(lambda x: unique_sorted_tuple([x['self_inference'], x['genderize.io'], x['chatpgt gender predictor']]), axis=1)


,combination,count,proportion
0,"((m, 2), (unknown, 1))",30,0.050000
1,"((f, 1), (m, 2))",20,0.033333
2,"((f, 2), (m, 1))",10,0.016667
3,"((f, 2), (unknown, 1))",9,0.015000
4,"((m, 2), (m , 1))",8,0.013333
5,"((m, 1), (unknown, 2))",6,0.010000
6,"((f, 1), (m, 1), (unknown, 1))",6,0.010000
7,"((f, 1), (unknown, 2))",5,0.008333


In [7]:
# Drop row 4 that has combination ((m, 2), (m , 1)) - not a disparity, all male
unique_combinations = unique_combinations.drop(4).reset_index(drop=True)
total_count = unique_combinations['count'].sum()
print('number of disparities out of 600 is ',total_count,'and the in terms of proportion, it is ',total_count/600)
unique_combinations

number of disparities out of 600 is  86 and the in terms of proportion, it is  0.14333333333333334


,combination,count,proportion
0,"((m, 2), (unknown, 1))",30,0.050000
1,"((f, 1), (m, 2))",20,0.033333
2,"((f, 2), (m, 1))",10,0.016667
3,"((f, 2), (unknown, 1))",9,0.015000
4,"((m, 1), (unknown, 2))",6,0.010000
5,"((f, 1), (m, 1), (unknown, 1))",6,0.010000
6,"((f, 1), (unknown, 2))",5,0.008333


#### What can be inferred from this?
1. most common disparity is of 2 tools classifying a name as male and one of them being uncertain with "unknown". An example of this with the names: Hiepnguyen, Maq, Never Hide, Mr Leaw, Anabadya Neogi, Rowun Giles. For these, on looking closely, they can be voted to be male

2. The presence of "unknown" in multiple combinations indicates inconsistency in gender identification across different sources. This shows that some names might be challenging to classify, possibly due to their ambiguity or cross-gender usage.

3. The combinations indicate various levels of disagreement between the sources, though the most common involves two agreeing on a specific gender and one differing. Thus, majority voting seems like the appropriate choice

4. There are only 3 cases where all 3 gender tools disagree, such for names like: Anh Le

In [8]:
df_first_600
# Count unique values in genderize.io column BEFORE threshold
genderize_io_counts = df_first_600['genderize.io'].value_counts()
genderize_io_counts

genderize.io
m          400
f          158
unknown     42
Name: count, dtype: int64

### MAJORITY VOTING 
First I will set a threshold of 0.85 that will change the male values with a certainity (probability score) below 0.85 to unknown. 

and then for majority voting:
1. Count Votes: Count the number of votes each gender received from the three sources.
2. Majority Decision: Choose the gender that received the majority of the votes. If one gender received two or more votes, assign that gender.
3. Tie-breaking: If each gender received one vote, you can either:
a. Use a Tie-breaker: Decide which source to trust more based on their overall accuracy across all names.

b. Use “Unknown”: If you want to avoid making arbitrary choices, you can assign the label "Unknown" when there is no majority.

and the gender guesser lib is much much less accurate so excluding it from consideration as it will throw off the other predictions

In [ ]:

'''
# Apply a condition to update 'm' to 'unknown' if the Gender Probability is below 0.85
df_first_600.loc[
    (df_first_600['genderize.io'] == 'm') & 
    (df_first_600['Gender Probability'] < 0.85), 
    'genderize.io'] = 'unknown'
df_first_600

'''

In [9]:
# Count unique values in genderize.io column after threshold set 
genderize_io_counts = df_first_600['genderize.io'].value_counts()
genderize_io_counts
#df_first_600

genderize.io
m          400
f          158
unknown     42
Name: count, dtype: int64

In [10]:
from collections import Counter

# Create a function for majority voting
def majority_vote(row):
    # Count votes
    counts = Counter(row)
    most_common = counts.most_common()
    
    # Majority Decision: if one gender received two or more votes, assign that gender
    if most_common[0][1] >= 2:
        return most_common[0][0]
    
    # Tie-breaking: if each gender received one vote
    if len(most_common) == 3:
        # Case (a): Use "Unknown"
        if "unknown" in counts:
            return "unknown"
        # Case (b): Use Tie-breaker (based on accuracy)
        else:
            return "unknown"
    else:
        return most_common[0][0]  # fallback case (unlikely to happen)

# Apply majority voting to the DataFrame
df_first_600['majority_gender'] = df_first_600.apply(
    lambda row: majority_vote([row['self_inference'], row['genderize.io'], row['chatpgt gender predictor']]), axis=1)

# Show resulting DataFrame
df_first_600.head(10)


/var/folders/_f/7jgfw7qx5jn8xl64g_9px9g00000gp/T/ipykernel_66533/2363627619.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_600['majority_gender'] = df_first_600.apply(


,cleaned_names,self_inference,genderize.io,chatpgt gender predictor,Gender Probability,majority_gender
0,Craig Hiller,m,m,m,1.00,m
1,Andrew Sheleby,m,m,m,1.00,m
2,Evan Ho,m,m,m,1.00,m
3,Brian Lam,m,m,m,1.00,m
4,Lin Peng,f,m,m,0.99,m
5,Denis Mukhin,m,m,m,0.97,m
6,Farnaz Fooladi,f,f,f,1.00,f
7,Farnad Jahanshah,m,m,m,1.00,m
8,Jason Seibel,m,m,m,1.00,m
9,Cathy Jiao,f,f,f,1.00,f


In [11]:
# Count unique values in majority_gender column
majority_gender_counts = df_first_600['majority_gender'].value_counts()
majority_gender_counts
#df_first_600

majority_gender
m          408
f          156
unknown     36
Name: count, dtype: int64

Did it on the first 600 using 3 tools and majority voting but 2 points to make here:
1. cannot manually do it for the entire dataset as it is too huge
2. chat gpt's predictor cannot take that many requests either, manually need to put it in and process
Thus, decision to just use genderize.io predictions henceforth and mention the caveat in the paper that it is not 100% accurate

In [12]:
# going to adpot the majority gender from the df above into the participants dataframe and the remaining genders from genderize.io
cleaned_names_only_with_genders = pd.read_csv(r'/Users/Isha/Desktop/TUe/Year 3/BEP/BEP_project/data/cleaned_names_only_with_genders.csv', sep=';')

cleaned_names_only_with_genders

,Unnamed: 0,cleaned_names,Gender,Gender Probability
0,0,Craig Hiller,male,1.00
1,1,Andrew Sheleby,male,1.00
2,2,Evan Ho,male,1.00
3,3,Brian Lam,male,1.00
4,4,Lin Peng,male,0.99
...,...,...,...,...
56490,56490,Mike Dimitrov,male,1.00
56491,56491,Baiyun ( Ben) Gao,female,0.70
56492,56492,Everet Siew,male,1.00
56493,56493,Amulya Garimella,female,0.99


In [13]:
# replacing male, female, and unknown with m,f, unknown for consistent formatting 
cleaned_names_only_with_genders['Gender'] = cleaned_names_only_with_genders['Gender'].replace(
    {'male': 'm', 'female': 'f', 'unknown': 'unknown'})

del cleaned_names_only_with_genders['Unnamed: 0']

cleaned_names_only_with_genders

,cleaned_names,Gender,Gender Probability
0,Craig Hiller,m,1.00
1,Andrew Sheleby,m,1.00
2,Evan Ho,m,1.00
3,Brian Lam,m,1.00
4,Lin Peng,m,0.99
...,...,...,...
56490,Mike Dimitrov,m,1.00
56491,Baiyun ( Ben) Gao,f,0.70
56492,Everet Siew,m,1.00
56493,Amulya Garimella,f,0.99


In [14]:
# compare the first 600 rows of df_first_600 with the first 600 rows of cleaned_names_only_with_genders to check if they are different or not, and how many are different 

# Check if the lengths of both dataframes are at least 600
if len(df_first_600) >= 600 and len(cleaned_names_only_with_genders) >= 600:
    # Extract the first 600 rows
    df_first_600_subset = df_first_600.iloc[:600]
    cleaned_names_only_with_genders_subset = cleaned_names_only_with_genders.iloc[:600]
    
    # Compare the 'majority_gender' column with the 'Gender' column
    differences = df_first_600_subset['majority_gender'] != cleaned_names_only_with_genders_subset['Gender']
    
    # Extract the rows that are different
    differing_rows = df_first_600_subset[differences].copy()
    differing_rows['Gender_in_cleaned_names_only_with_genders'] = cleaned_names_only_with_genders_subset['Gender'][differences]
    # differing_rows = differing_rows[['cleaned_names', 'majority_gender', 'Gender_in_cleaned_names_only_with_genders']]
    differing_rows[['cleaned_names', 'majority_gender', 'Gender_in_cleaned_names_only_with_genders']]
    
    # Count the number of discrepancies
    num_differences = differences.sum()

else:
    differing_rows = pd.DataFrame(columns=['cleaned_names', 'majority_gender', 'Gender_in_cleaned_names_only_with_genders'])
    num_differences = "One or both DataFrames have less than 600 rows."

print(num_differences) # number of rows that are different from majority voting and genderize.io
differing_rows # showings the value of the differing rows 


40


,cleaned_names,self_inference,genderize.io,chatpgt gender predictor,Gender Probability,majority_gender,Gender_in_cleaned_names_only_with_genders
13,Yun Chuan Ngin,m,f,m,0.55,m,f
18,Crashnb,unknown,m,unknown,1.00,unknown,m
23,Edwinwu,m,unknown,m,0.00,m,unknown
40,A Jelinsky,unknown,m,unknown,0.68,unknown,m
42,Euann Gu,unknown,f,m,0.50,unknown,f
57,Ultrafire X Arunmolithevar,m,unknown,m,0.00,m,unknown
69,Manuel Dionne,m,f,m,0.95,m,f
71,Anamaria Nicola,f,m,f,0.83,f,m
83,Neelavarsha D K,f,unknown,f,0.00,f,unknown
91,Mrunal Puram,f,m,f,0.67,f,m


In [15]:
# if the gender probability column is not there above then run this:
'''
# Merge the DataFrame df with cleaned_names_only_with_genders to add the 'Gender Probability' column correctly
# avoid adding the column twice
differing_rows = pd.merge(differing_rows, cleaned_names_only_with_genders[['cleaned_names', 'Gender Probability']],
              on='cleaned_names', how='left', suffixes=('', '_drop'))

# Drop any duplicate columns created by the merge operation
differing_rows.drop([col for col in differing_rows.columns if 'drop' in col], axis=1, inplace=True)

# Display the updated DataFrame to check if the operation was successful
differing_rows
'''


differing_rows[['cleaned_names','majority_gender','genderize.io','Gender Probability']]

,cleaned_names,majority_gender,genderize.io,Gender Probability
13,Yun Chuan Ngin,m,f,0.55
18,Crashnb,unknown,m,1.00
23,Edwinwu,m,unknown,0.00
40,A Jelinsky,unknown,m,0.68
42,Euann Gu,unknown,f,0.50
57,Ultrafire X Arunmolithevar,m,unknown,0.00
69,Manuel Dionne,m,f,0.95
71,Anamaria Nicola,f,m,0.83
83,Neelavarsha D K,f,unknown,0.00
91,Mrunal Puram,f,m,0.67


In [16]:
# redoing and checking for disparity distribution in the differing rows
# Create a function to represent the unique sorted tuple
def unique_sorted_tuple(row):
    # Extract the tuple
    unique_comb = tuple(sorted(Counter(row).items()))
    return unique_comb

# Filter out rows where all values are the same
disparities = differing_rows[
    (differing_rows['self_inference'] != differing_rows['genderize.io']) |
    (differing_rows['self_inference'] != differing_rows['chatpgt gender predictor']) |
    (differing_rows['genderize.io'] != differing_rows['chatpgt gender predictor'])
]

# Apply unique sorted tuple function
disparities['unique_comb'] = disparities.apply(lambda x: unique_sorted_tuple([x['self_inference'], x['genderize.io'], x['chatpgt gender predictor']]), axis=1)

# Get unique combinations and their counts
unique_combinations = disparities['unique_comb'].value_counts().reset_index()
unique_combinations.columns = ['combination', 'count']

# Calculate the proportions
unique_combinations['proportion'] = unique_combinations['count'] / len(df_first_600)

# Display results
unique_combinations

,combination,count,proportion
0,"((m, 2), (unknown, 1))",10,0.016667
1,"((f, 1), (m, 2))",9,0.015000
2,"((f, 2), (unknown, 1))",7,0.011667
3,"((f, 1), (m, 1), (unknown, 1))",6,0.010000
4,"((m, 1), (unknown, 2))",4,0.006667
5,"((f, 2), (m, 1))",3,0.005000
6,"((f, 1), (unknown, 2))",1,0.001667


### Checking for missclassifcation rates:
FN means classifying a participant as male when they are actually female. So first I will find how many of those kinds of cases exist from the 40 disparities found above and then will set a probability threshold for furture rows

In [17]:
# Filter rows where majority gender is 'f', but genderize.io classified as 'm' or 'unknown'
female_misclassified = df_first_600[
    (df_first_600['majority_gender'] == 'f') & 
    ((df_first_600['genderize.io'] == 'm') | (df_first_600['genderize.io'] == 'unknown'))
]

# Include the gender probability in the filtered DataFrame
female_misclassified = female_misclassified[['cleaned_names', 'majority_gender', 'genderize.io', 'Gender Probability']]

print("Number of females misclassified as males or unknown:", len(female_misclassified))
female_misclassified

Number of females misclassified as males or unknown: 10


,cleaned_names,majority_gender,genderize.io,Gender Probability
71,Anamaria Nicola,f,m,0.83
83,Neelavarsha D K,f,unknown,0.00
91,Mrunal Puram,f,m,0.67
106,Scarlatte,f,unknown,0.00
126,Arianazhu,f,unknown,0.00
153,Emilyemilyb,f,unknown,0.00
203,Nicola Thouliss,f,m,0.83
308,Ashnajay,f,unknown,0.00
341,Alarissamoreira,f,unknown,0.00
566,Faisha Surjatin,f,unknown,0.00


#### Result:

**BEFORE THRESHOLD:**
40 out of 600 from genderize.io are different from the majority voted gender. And of these 40 rows, only 3 females are missclassifed as males and 7 as unknown. Considering that, after the first 600 rows, genderize.io will be the primary gender classification tool, if we choose to direct our focus on reducing these false negatives then we need to somehow set a threshold on the probability. 

**AFTER THRESHOLD:**
52 out of 600 from genderize.io are different from the majority voted gender. And of these 52 rows, 10 females are still missclassifed but all as 'unknown'. 

In [18]:
# Filter rows where both majority gender and genderize.io are 'f'
female_true_positives = df_first_600[
    (df_first_600['majority_gender'] == 'f') & 
    (df_first_600['genderize.io'] == 'f')
]

# Include the gender probability in the filtered DataFrame
female_true_positives = female_true_positives[['cleaned_names', 'majority_gender', 'genderize.io', 'Gender Probability']]

print("Number of female true positives:", len(female_true_positives))
female_true_positives

Number of female true positives: 146


,cleaned_names,majority_gender,genderize.io,Gender Probability
6,Farnaz Fooladi,f,f,1.00
9,Cathy Jiao,f,f,1.00
12,Maria Mangru Mangru,f,f,1.00
15,Mariam Singer,f,f,1.00
16,Bonnie Li,f,f,1.00
...,...,...,...,...
578,Vee Dev,f,f,0.59
588,Lei Lei,f,f,0.55
589,Charity Kehl,f,f,1.00
591,Jeanette Pranin,f,f,1.00


In [19]:
# Filter rows where both majority gender and genderize.io are 'f'
male_true_negatives = df_first_600[
    (df_first_600['majority_gender'] == 'm') & 
    (df_first_600['genderize.io'] == 'm')
]

# Include the gender probability in the filtered DataFrame
male_true_negatives = male_true_negatives[['cleaned_names', 'majority_gender', 'genderize.io', 'Gender Probability']]

print("Number of male true negatives:", len(male_true_negatives))
male_true_negatives

Number of male true negatives: 389


,cleaned_names,majority_gender,genderize.io,Gender Probability
0,Craig Hiller,m,m,1.00
1,Andrew Sheleby,m,m,1.00
2,Evan Ho,m,m,1.00
3,Brian Lam,m,m,1.00
4,Lin Peng,m,m,0.99
...,...,...,...,...
593,Ralph Hubbard,m,m,1.00
594,Samir Borle,m,m,0.99
595,Ali Malik,m,m,0.94
598,Gautham Kovvuru,m,m,1.00


In [20]:
# Filter rows where both majority gender and genderize.io are 'f'
unknown_true_negatives = df_first_600[
    (df_first_600['majority_gender'] == 'unknown') & 
    (df_first_600['genderize.io'] == 'unknown')
]

# Include the gender probability in the filtered DataFrame
unknown_true_negatives = unknown_true_negatives[['cleaned_names', 'majority_gender', 'genderize.io', 'Gender Probability']]

print("Number of unknown true negatives:", len(unknown_true_negatives))
unknown_true_negatives

Number of unknown true negatives: 25


,cleaned_names,majority_gender,genderize.io,Gender Probability
29,Caradr,unknown,unknown,0.0
84,Huangkellyb,unknown,unknown,0.0
88,Gibbsworthy,unknown,unknown,0.0
94,Alimbabson,unknown,unknown,0.0
120,Gvlastar,unknown,unknown,0.0
162,Noodleleg,unknown,unknown,0.0
311,Kongcho,unknown,unknown,0.0
328,Darcychung,unknown,unknown,0.0
349,Czhao,unknown,unknown,0.0
371,Bryanso,unknown,unknown,0.0


In [21]:
# Filter rows where genderize.io is 'f' but majority_gender is 'm'
male_false_positives = df_first_600[
    (df_first_600['genderize.io'] == 'f') & 
    (df_first_600['majority_gender'] == 'm')
]

# Select relevant columns to display if needed, here displaying all for clarity
male_false_positives = male_false_positives[['cleaned_names', 'majority_gender', 'genderize.io', 'Gender Probability']]

print("Number of false positives:", len(male_false_positives))
male_false_positives

Number of false positives: 9


,cleaned_names,majority_gender,genderize.io,Gender Probability
13,Yun Chuan Ngin,m,f,0.55
69,Manuel Dionne,m,f,0.95
139,Zovin Chong,m,f,1.00
144,Dang Quan Anh Mai,m,f,0.97
255,Irvino Djuana,m,f,0.99
284,Nicolas Mac Beth,m,f,1.00
312,Hak Soo Kim,m,f,0.70
321,Aydin Gokce,m,f,0.82
370,Nafis Abeer,m,f,0.92


In [22]:
# Filter rows where genderize.io is 'f' but majority_gender is 'unknown'
unknown_false_positives = df_first_600[
    (df_first_600['genderize.io'] == 'f') & 
    (df_first_600['majority_gender'] == 'unknown')
]

# Select relevant columns to display if needed, here displaying all for clarity
unknown_false_positives = unknown_false_positives[['cleaned_names', 'majority_gender', 'genderize.io', 'Gender Probability']]

print("Number of false positives:", len(unknown_false_positives))
unknown_false_positives

Number of false positives: 3


,cleaned_names,majority_gender,genderize.io,Gender Probability
42,Euann Gu,unknown,f,0.50
234,Grape,unknown,f,0.74
447,Johnary Kim,unknown,f,0.70


In [23]:
# Find rows not fitting in TP, FP, TN, FN
all_conditions = (
    ((df_first_600['genderize.io'] == 'f') & (df_first_600['majority_gender'] == 'f')) |  # TP
    ((df_first_600['genderize.io'] == 'f') & (df_first_600['majority_gender'] != 'f')) |  # FP
    ((df_first_600['genderize.io'] != 'f') & (df_first_600['majority_gender'] != 'f')) |  # TN
    ((df_first_600['genderize.io'] != 'f') & (df_first_600['majority_gender'] == 'f'))    # FN
)

# Filter for rows that do not match any condition
missing_classifications = df_first_600[~all_conditions]

print("Number of missing classifications:", len(missing_classifications))
missing_classifications

Number of missing classifications: 0


,cleaned_names,self_inference,genderize.io,chatpgt gender predictor,Gender Probability,majority_gender


### Going further and updating the rest of the dataset:

In [24]:
# update the first 600 values of the majority_gender column from df_first_600 into the Gender column of cleaned_names_only_with_genders and retain the remaining rows as they are

# Ensure both DataFrames have at least 600 rows
if len(df_first_600) >= 600 and len(cleaned_names_only_with_genders) >= 600:
    # Update the first 600 rows
    cleaned_names_only_with_genders.loc[:599, 'Gender'] = df_first_600['majority_gender'].iloc[:600]

In [25]:
cleaned_names_only_with_genders.iloc[596] # checking specific rows to check if it worked correctly
cleaned_names_only_with_genders.iloc[569]
cleaned_names_only_with_genders.iloc[336]
cleaned_names_only_with_genders.iloc[137]
cleaned_names_only_with_genders.iloc[57]

cleaned_names         Ultrafire X Arunmolithevar
Gender                                         m
Gender Probability                           0.0
Name: 57, dtype: object

In [26]:
# Count unique values in majority_gender column
cleaned_gender_counts = cleaned_names_only_with_genders['Gender'].value_counts()
cleaned_gender_counts

Gender
m          39617
f          13684
unknown     3194
Name: count, dtype: int64

In [27]:
#add the 'Gender' and 'Gender Probability' columns from cleaned_names_only_with_genders into participants_links_expanded

# Add 'Gender' column
participants_links_expanded['Gender'] = cleaned_names_only_with_genders['Gender'] # has the first 600 from majority voting 
participants_links_expanded['genderize.io'] = cleaned_names_only_with_genders['Gender'] # has all from genderize.io

# Add 'Gender Probability' column
participants_links_expanded['Gender Probability'] = cleaned_names_only_with_genders['Gender Probability']
participants_links_expanded['Genderize.io Probability'] = cleaned_names_only_with_genders['Gender Probability']

# Display the resulting DataFrame
participants_links_expanded.head()

,Unnamed: 0,url,name,website,github,twitter,address,skills,interests,projects,hackathons,cleaned_names,Gender,genderize.io,Gender Probability,Genderize.io Probability
0,0,https://devpost.com/CraigHiller,Craig Hiller,NaN,NaN,NaN,NaN,NaN,NaN,https://devpost.com/software/googme,https://calhacks.devpost.com/,Craig Hiller,m,m,1.00,1.00
1,1,https://devpost.com/asheleby,Andrew Sheleby,NaN,NaN,NaN,United States,wordpress|python|tableau|excel|Blockchain|,AR/VR|Blockchain|Communication|Cybersecurity|D...,https://devpost.com/software/helpr-27tjik,https://hackhealth-2021.devpost.com/,Andrew Sheleby,m,m,1.00,1.00
2,2,https://devpost.com/evanho239,Evan Ho,NaN,NaN,NaN,United States,java|c++|javascript|angular.js|css|html|Blockc...,Beginner Friendly|Blockchain|Cybersecurity|Des...,https://devpost.com/software/untitled-for-now,https://hack32022.devpost.com/,Evan Ho,m,m,1.00,1.00
3,3,https://devpost.com/brilamb,Brian Lam,https://github.com/Brooseidon,https://github.com/Brooseidon,NaN,US,java|javascript|,Communication|Cybersecurity|Gaming|IoT|Machine...,https://devpost.com/software/the-hummingbird-p...,"https://hacksc-2020.devpost.com/,https://sd-ha...",Brian Lam,m,m,1.00,1.00
4,4,https://devpost.com/linpeng06,lin peng,https://github.com/linpeng06,https://github.com/linpeng06,NaN,NaN,android|javascript|php|java|ios|jquery|css|htm...,NaN,https://devpost.com/software/arduinosetup,https://pearlhacks2016.devpost.com/,Lin Peng,m,m,0.99,0.99


In [28]:
#del participants_links_expanded['gender_guesser_lib']
participants_links_expanded[['cleaned_names','Gender','Gender Probability']][:20]

,cleaned_names,Gender,Gender Probability
0,Craig Hiller,m,1.00
1,Andrew Sheleby,m,1.00
2,Evan Ho,m,1.00
3,Brian Lam,m,1.00
4,Lin Peng,m,0.99
5,Denis Mukhin,m,0.97
6,Farnaz Fooladi,f,1.00
7,Farnad Jahanshah,m,1.00
8,Jason Seibel,m,1.00
9,Cathy Jiao,f,1.00


### Alex says use gender probability to reduce false negatives

Setting a threshold of 0.85 below which the 'm' values should be changed to 'unknown'.

In [ ]:
'''
# Apply a condition to update 'm' to 'unknown' if the Gender Probability is below 0.85
participants_links_expanded.loc[
    (participants_links_expanded['Gender'] == 'm') & 
    (participants_links_expanded['Gender Probability'] < 0.85), 
    'Gender'] = 'unknown'
'''

In [29]:
participants_links_expanded[['cleaned_names','Gender','Gender Probability']][:20]

,cleaned_names,Gender,Gender Probability
0,Craig Hiller,m,1.00
1,Andrew Sheleby,m,1.00
2,Evan Ho,m,1.00
3,Brian Lam,m,1.00
4,Lin Peng,m,0.99
5,Denis Mukhin,m,0.97
6,Farnaz Fooladi,f,1.00
7,Farnad Jahanshah,m,1.00
8,Jason Seibel,m,1.00
9,Cathy Jiao,f,1.00


In [9]:
# Count unique values in majority_gender column
cleaned_gender_counts_post_threshold = participants_links_expanded['Gender'].value_counts()
cleaned_gender_counts_post_threshold

KeyError: 'Gender'

In [30]:
participants_links_expanded.to_csv('participants_expanded_post_inference.csv')